In [1]:
# Requirements dosyasında belirtilen ve programda kullanılan kütüphanelerin yüklenmesi
import subprocess

# requirements.txt dosyasının adı
requirements_file = "requirements.txt"

# requirements.txt dosyasındaki kütüphaneleri yükleme
try:
    subprocess.run(["pip", "install", "-r", requirements_file])
    print("Kütüphaneler başarıyla yüklendi.")
except Exception as e:
    print(f"Hata oluştu: {e}")


Kütüphaneler başarıyla yüklendi.


In [2]:
# bütün kullanılacak kütüphaneler burada import edilir
import os
import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from transformers import AutoTokenizer, AutoModel
import torch
from gensim.models import FastText
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# nltk kütüphanesindeki stopwords ve noktalama işaretleri listesi indirilir
nltk.download('stopwords')
nltk.download('punkt')


c:\Users\Sadi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


False

In [3]:
# NLP ile metin önişleme fonksiyonu
def text_processing(text):
    stop_words = set(stopwords.words('english')) # stop words seçilir
    word_tokens = word_tokenize(text) # metindeki kelimeler ayrıştırılır
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words] #stop words çıkarılır
    stemmer = PorterStemmer() # noktalama işaretleri çıkarılır
    stemmed_words = [stemmer.stem(word) for word in filtered_text] 
    result = " ".join(stemmed_words)

    return result

In [4]:
# daha önceden eğitilimiş scibert modeli yüklenir
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased") 
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

c:\Users\Sadi\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# scibert vektör alma fonksiyonu
def get_scibert_vector(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

    return embeddings

In [6]:
# scibert vektörlerini csv olarak kaydeden fonksiyon
def save_scibert_embedding_as_csv(name,embeddings):

    file_name = "scibert_embeddings/"+name+".csv"

    with open(file_name, "w") as file:
        for embedding in embeddings:
            embedding_str = ",".join([str(value.item()) for value in embedding])
            file.write(embedding_str + "\n")

    print(f"Embeddings dosyası '{file_name}' olarak kaydedildi.")

In [42]:
# fasttext ile vektör alma fonksiyonu
def get_fastxt_vector(text):
    # Metinleri tokenize etme
    tokenized_texts = simple_preprocess(text)

    # FastText modelini eğitme
    model = FastText(tokenized_texts, window=5, min_count=1)

    # Her bir metnin vektör temsilini alıyoruz
    vector_embeddings = [model.wv[text] for text in tokenized_texts]
    return vector_embeddings
    # Her bir metnin vektör temsilini yazdırma
    #for i, embedding in enumerate(vector_embeddings):
        #print(f"Metin {i+1} vektör temsili: {embedding}")


In [8]:
# fasttext vektörlerini csv olarak kaydeden fonksiyon
def save_fast_embedding_as_csv(name,embeddings):

    file_name = "fasttext_embeddings/"+name+".csv"

    with open(file_name, "w") as file:
        for embedding in embeddings:
            embedding_str = ",".join([str(value.item()) for value in embedding])
            file.write(embedding_str + "\n")

    print(f"Embeddings dosyası '{file_name}' olarak kaydedildi.")

In [43]:
# tüm makalelerin vektörlerinin çıkarılıp csv olarak kaydedilmesi (NOT: tek seferlik çalıştırılır. Çok vakit almaktadır)
# makalelerin bulunduğu klasörlerin yolu
metin_klasor = "Inspec/docsutf8" 
key_klasor = "Inspec/keys"

# Klasördeki dosyaları listele
dosyalar = os.listdir(metin_klasor)

i = 0

# Her dosya için işlem yap
for dosya in dosyalar:
    
    key = dosya[:-4]+'.key'
    # Dosya yolunu oluştur
    dosya_yolu = os.path.join(metin_klasor, dosya)
    # Key yolunu oluştur
    key_yolu = os.path.join(key_klasor, key)

    # Dosyanın bir text dosyası olduğundan emin ol
    if os.path.isfile(dosya_yolu) and dosya.endswith(".txt"):
        # Dosyayı aç ve içeriğini oku
        with open(dosya_yolu, "r", encoding="utf-8") as dosya_objesi:
            # Dosyanın içeriğini oku
            dosya_icerigi = dosya_objesi.read()
            stemmed_txt = text_processing(dosya_icerigi) # metin ön işleme 
            scibert_embedding = get_scibert_vector(stemmed_txt) # scibert vektörlerini çıkart
            save_scibert_embedding_as_csv(dosya[:-4],scibert_embedding) # scibert vektörlerini csv olarak kaydet
            
            fast_embedding = get_fastxt_vector(stemmed_txt) # fasttext vektörlerini çıkart
            save_fast_embedding_as_csv(dosya[:-4],fast_embedding) # fasttext vektörlerini csv olarak kaydet
            #print(stemmed_txt)
        #with open(key_yolu, "r", encoding="utf-8") as key_objesi:
            # Dosyanın içeriğini oku
            #key_icerigi = key_objesi.read()
            #print(key)
            #print(key_icerigi)
            #text_processing(key_icerigi)
    #i += 1
    #if i ==3:


Embeddings dosyası 'scibert_embeddings/100.csv' olarak kaydedildi.
22
Embeddings dosyası 'fasttext_embeddings/100.csv' olarak kaydedildi.


In [54]:
# Cosine Similarity ile kullanıcı profili ile makaleler arasında benzerlik hesaplama fonksiyonu
def calculate_similarity(user_profile, article_vectors):
    similarities = cosine_similarity(user_profile, article_vectors)
    return similarities[0]

# fasttext için başlangıç öneri fonksiyonu
def recommend_articles_fasttext(user_profile_fasttext, article_vectors_fasttext, top_n=5):
    similarities_fasttext = calculate_similarity(user_profile_fasttext, article_vectors_fasttext)
    sorted_indices_fasttext = similarities_fasttext.argsort()[::-1][:top_n]
    recommended_articles_fasttext = [(index, similarities_fasttext[index]) for index in sorted_indices_fasttext]
    return recommended_articles_fasttext

# fasttext için makale seçimi sonrası öneri fonksiyonu
def recommend_article_2_article_fasttext(user_profile_fasttext, article_vectors_fasttext, top_n=5):
    similarities_fasttext = []
    for user_vector in user_profile_fasttext:
        similarities = cosine_similarity(user_vector, article_vectors_fasttext)[0]
        similarities_fasttext.append(similarities)
    
    similarities_fasttext = np.mean(similarities_fasttext, axis=0)
    
    sorted_indices_fasttext = similarities_fasttext.argsort()[::-1][:top_n]
    recommended_articles_fasttext = [(index, similarities_fasttext[index]) for index in sorted_indices_fasttext]
    return recommended_articles_fasttext

# scibert için öneri fonksiyonu
def recommend_articles_scibert(user_profile_scibert, article_vectors_scibert, top_n=5):
    similarities_scibert = calculate_similarity(user_profile_scibert, article_vectors_scibert)
    sorted_indices_scibert = similarities_scibert.argsort()[::-1][:top_n]
    recommended_articles_scibert = [(index, similarities_scibert[index]) for index in sorted_indices_scibert]
    return recommended_articles_scibert

In [11]:
# kullanıcı ilgi alanlarının scibert vektörlerini çıkarıp ortalamasını alan fonksiyon
def get_average_scibert(text_list):
    vectors = []
    for text in text_list:
        stemmed_txt = text_processing(text)
        vector = get_scibert_vector(stemmed_txt)
        vectors.append(vector)
    
    # Vektörlerin ortalamasını al
    average_vector = torch.stack(vectors).mean(dim=0).detach().numpy()
    
    return average_vector


In [12]:
# kullanıcı ilgi alanlarının fasttext vektörlerini çıkarıp ortalamasını alan fonksiyon
def get_average_fastxt(text_list):
    vectors = []
    for text in text_list:
        stemmed_txt = text_processing(text)
        vector = get_fastxt_vector(stemmed_txt)
        vectors.append(vector)
    
    # Vektörlerin ortalamasını al
    average_vector = np.mean(vectors, axis=0)
    average_vector = [sum(sublist) / len(sublist) for sublist in zip(*vectors)]    
    return average_vector

In [44]:
def read_embeddings_from_csv_folder(folder_path):
    all_embeddings = []
    file_names = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            embeddings, file_name = read_embeddings_from_csv(file_path)
            all_embeddings.extend(embeddings)
            for i in range(len(embeddings)):
                file_names.append(file_name)
    return all_embeddings, file_names

def read_embeddings_from_csv(file_path):
    embeddings = []
    with open(file_path, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            embeddings.append([float(value) for value in row])
    return embeddings, os.path.basename(file_path)

In [45]:
# fasttext ve scibert embedding dosyalarını okur
folder_path = "fasttext_embeddings"
article_vectors_fasttext, fasttext_file_names = read_embeddings_from_csv_folder(folder_path)

folder_path = "scibert_embeddings"
article_vectors_scibert, scibert_file_names = read_embeddings_from_csv_folder(folder_path)

In [48]:
# Örnek kullanıcı profili 
# kullanıcı ilgi alanları seçer
user_profile = ["Computer engineering", "system design", "network"]

# kullanıcı ilgi alanlarının scibert ve fasttext ile vektörleri hesaplanıp tüm vektörlerin ortalaması alınır
user_profile_scibert = get_average_scibert(user_profile)           
user_profile_fasttext = get_average_fastxt(user_profile)

# FastText için 5 öneri sunulur
print("FastText ile öneriler:")
for index, similarity in recommend_articles_fasttext(user_profile_fasttext, article_vectors_fasttext):
    print(f"Makale {fasttext_file_names[index]}, Benzerlik: {similarity}")

# SciBERT için 5 öneri sunulur
print("\nSciBERT ile öneriler:")
for index, similarity in recommend_articles_scibert(user_profile_scibert, article_vectors_scibert):
    print(f"Makale {scibert_file_names[index]}, Benzerlik: {similarity}")

2
2
1
FastText ile öneriler:
Makale 4.csv, Benzerlik: 0.5943362409095669
Makale 271.csv, Benzerlik: 0.5943362409095669
Makale 1990.csv, Benzerlik: 0.5943362409095669
Makale 1990.csv, Benzerlik: 0.5943362409095669
Makale 1990.csv, Benzerlik: 0.5943362409095669

SciBERT ile öneriler:
Makale 218.csv, Benzerlik: 0.7906550644234528
Makale 1959.csv, Benzerlik: 0.7853924203847588
Makale 1455.csv, Benzerlik: 0.7717146099828255
Makale 1543.csv, Benzerlik: 0.771253393736498
Makale 1736.csv, Benzerlik: 0.7646994766367001


c:\Users\Sadi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


In [63]:
# Eğer makale seçimi yapılırsa
# seçilen makaleye benzer olan makale bulunup en yakın makale önerilir

selected_article = "4"

user_prefer_fast, file = read_embeddings_from_csv("fasttext_embeddings/"+selected_article+".csv")
recommended_fast_articles = []
# FastText için 5 öneri sunulur
print("FastText ile öneriler:")
for index, similarity in recommend_articles_fasttext(user_prefer_fast, article_vectors_fasttext):
    print(f"Makale {fasttext_file_names[index]}, Benzerlik: {similarity}")
    recommended_fast_articles.extend(fasttext_file_names[index])

user_prefer_scibert, file = read_embeddings_from_csv("scibert_embeddings/"+selected_article+".csv")

# Scibert için 5 öneri sunulur
print("scibert ile öneriler:")
for index, similarity in recommend_articles_scibert(user_prefer_scibert, article_vectors_scibert):
    print(f"Makale {fasttext_file_names[index]}, Benzerlik: {similarity}")

FastText ile öneriler:
Makale 344.csv, Benzerlik: 1.0000000000000004
Makale 1708.csv, Benzerlik: 1.0000000000000004
Makale 2160.csv, Benzerlik: 1.0000000000000004
Makale 1686.csv, Benzerlik: 1.0000000000000004
Makale 1708.csv, Benzerlik: 1.0000000000000004
scibert ile öneriler:
Makale 1016.csv, Benzerlik: 1.0000000000000016
Makale 1016.csv, Benzerlik: 0.8898809300869547
Makale 1008.csv, Benzerlik: 0.8851984271505704
Makale 1012.csv, Benzerlik: 0.8839402051680088
Makale 1004.csv, Benzerlik: 0.8819873122876638


In [62]:
def calculate_precision_recall(selected_articles, recommended_articles, top_n=5):
    # Seçilen makalelerin sayısı
    selected_count = len(selected_articles)
    
    # Tavsiye edilen makalelerin sayısı
    recommended_count = top_n
    
    # Doğru önerilen makalelerin sayısı
    correct_count = sum(1 for article in recommended_articles[:top_n] if article in selected_articles)
    
    # Precision hesapla
    precision = correct_count / recommended_count if recommended_count > 0 else 0
    
    # Recall hesapla
    recall = correct_count / selected_count if selected_count > 0 else 0
    
    return precision, recall

In [69]:
# Kullanıcı tarafından seçilen makaleler aşağuıdaki gibi olduğunu varsayalım
selected_articles = ["4","271"]

precision, recall = calculate_precision_recall(selected_articles, recommended_fast_articles)
print("Precision:", precision)
print("Recall:", recall)


Precision: 0.4
Recall: 1.0
